In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from pandas_profiling import ProfileReport

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_10844\1857573116.py:10: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [3]:
df = pd.read_csv("../data/interim/datos_limpios.csv", encoding="latin-1", sep=";")

## Analisis exporatorio previo al preprocesamiento

In [31]:
report = ProfileReport(df)
report.to_file("../reports/1.0-jrb-exploratory-data-analysis.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Analisis de Columna Codigo Diagnostico

- Esta columna carece de valores faltantes

In [8]:
largo_codigos = df["Código Diagnóstico"].str.len().value_counts(normalize=True, dropna=False)

In [9]:
largo_codigos

5    0.585981
3    0.270380
7    0.091945
2    0.019834
6    0.015504
4    0.010534
1    0.005819
8    0.000004
Name: Código Diagnóstico, dtype: float64

- El 58.5% de los codigos presenta largo 5. Los codigos con largo 5 son del tipo: "C38.4", por
lo que es el formato deseable

- El 27.0% de los codigos presenta largo 3. Los codigos con largo 3 son del tipo: "R05", "C34", por
lo que corresponde a la Categoria del diagnostico.

- El 9.20% de los codigos presenta largo 7. Los codigos con largo 7 son del tipo: "J44.8.4", por
lo que corresponde al diagnostico deseable expandido. A estos codigos se les puede
eliminar el ultimo digito y quedan en formato deseable

- El 1.98% de los codigos presenta largo 2. Los codigos con largo 2 son del tipo: "55", por lo que
tienen una notacion de diagnostico deficiente. Sin embargo, estos codigos se relacionan a un
unico "Nombre Diagnostico". Esto puede permitir recodificar estos codigos de forma mas facil.

In [18]:
df[df["Código Diagnóstico"].str.len() == 2][["Código Diagnóstico", "Nombre Diagnóstico"]].value_counts()

Código Diagnóstico  Nombre Diagnóstico                    
55                  Fibrosis pulmonar                         4335
79                  Trasplante  Pulmon                        2788
78                  Trasplante Cardiaco                       1197
41                  Nódulo pulmonar                            842
20                  Bronquiectasias                            363
54                  Enfermedad pulmonar difusa                 342
42                  Tumor pulmonar                             113
77                  Asma Bronquial en el Adulto (Sospecha)     110
74                  Tabaquismo                                 100
64                  Hipertensión pulmonar primaria              99
17                  Neumonitis por hipersensibilidad            98
49                  Disnea en estudio                           79
44                  Cáncer pulmonar                             53
21                  Bronquiectasias infectadas                  50
59 